In [2]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [4]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 12.8/12.8 MB 3.4 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [5]:
import spacy
import PyPDF2
import json

In [6]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

resume_text = extract_text_from_pdf("Harshal_Meshram_Resume.pdf")

In [7]:
resume_text

'Harshal Meshram\nćhmeshram141@gmail.com Ħ+91 8788415755 Ƣ2000-10-20 ]LinkedIn\nEDUCATION\nCENTRE FOR DEVELOPMENT OF ADVANCED COMPUTING Sep 2023 - Feb 2024\nBig Data Analytics (PG Diploma)\nSKN COLLEGE OF ENGINEERING Aug 2019 - Apr 2022\nMechanical Engineering (BE)\nTECHNICAL STACK\n•Skills: Python, Pandas, Linux, SQL, Advanced SQL, MYSQL, MongoDB, Hadoop, Hive, PySpark\n•Databases: Oracle, NOSQL\n•Coding Platforms: Jupyter Notebook, Google Colab, Visual Studio, Databricks\n•Basic Knowledge of AWS Services and Cloud Computing Concepts, Data Warehouse, Machine Learning, Tableau, Excel\nACADEMIC PROJECTS\nCustomer Churn Data Analysis For Telecom Services Jan 2024 - Feb 2024\nBuilt and automated an ETL pipeline using AWS S3, AWS Glue, Redshift & Airflow.\n•Extracted customer data from S3 buckets using crawlers and inferred schema.\n•Transformed and loaded data into a Redshift data warehouse for analysis.\n•Visualized churn insights through Power BI dashboards and reports.\n•Orchestrated a

In [8]:
nlp = spacy.load("en_core_web_sm")

In [10]:
doc = nlp(resume_text)

In [12]:
import re
import json
import PyPDF2

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

def extract_entities(resume_text):
    entities = {"name": "", "email": "", "phone": "", "date_of_birth": "", "education": []}

    # Extracting name
    name_pattern = r'^([A-Z][a-z]+)\s([A-Z][a-z]+)'
    match = re.search(name_pattern, resume_text)
    if match:
        entities["name"] = match.group(0)

    # Extracting email
    email_pattern = r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}'
    match = re.search(email_pattern, resume_text)
    if match:
        entities["email"] = match.group(0)

    # Extracting phone number
    phone_pattern = r'\b\d{10}\b'
    match = re.search(phone_pattern, resume_text)
    if match:
        entities["phone"] = match.group(0)

    # Extracting date of birth
    date_patterns = [
        r'(\d{4})-(\d{2})-(\d{2})',  # Year-Month-Day
        r'(\d{2})-(\d{2})-(\d{4})',  # Day-Month-Year
        r'(\d{2})-(\d{4})-(\d{2})',  # Day-Year-Month
        # Some Test Cases
    ]

    for pattern in date_patterns:
        match = re.search(pattern, resume_text)
        if match:
            if len(match.groups()) == 3:  # Check if all components are captured
                if len(match.group(1)) == 4:  # Check if the year is in the first group
                    year = int(match.group(1))
                    month = int(match.group(2))
                    day = int(match.group(3))
                else:  # If year is not in the first group, rearrange the groups
                    day = int(match.group(1))
                    month = int(match.group(2))
                    year = int(match.group(3))

                if day <= 12 and month <= 12:
                    # If both day and month are less than or equal to 12, consider the higher number as month
                    month = max(day, month)
                elif day <= 12:
                    # If day is less than or equal to 12, consider it as month
                    month = day

                entities["date_of_birth"] = f"{month}-{year}"
                break

    # Extracting education details
    education_pattern = r'EDUCATION(.*?)TECHNICAL STACK'
    match = re.search(education_pattern, resume_text, re.DOTALL)
    if match:
        education_details = match.group(1)
        college_pattern = r'([A-Za-z -]+)\s+(\w{3})\s(\d{4})\s-\s(\w{3})\s(\d{4})\s(.+)'
        matches = re.findall(college_pattern, education_details)
        for match in matches:
            college_name = match[0]
            degree_details = match[5]
            degree_pattern = r'(.+) \((.+)\)'
            degree_match = re.search(degree_pattern, degree_details)
            if degree_match:
                stream = degree_match.group(1)
                degree_type = degree_match.group(2)
                education = {"college_name": college_name, "stream": stream, "degree_type": degree_type}
                entities["education"].append(education)

    return entities

pdf_path = "Harshal_Meshram_Resume.pdf"
resume_text = extract_text_from_pdf(pdf_path)
entities = extract_entities(resume_text)

# Output the extracted entities in JSON format
with open("extracted_entities.json", "w") as outfile:
    json.dump(entities, outfile, indent=4)

print(json.dumps(entities, indent=4))

{
    "name": "Harshal Meshram",
    "email": "hmeshram141@gmail.com",
    "phone": "8788415755",
    "date_of_birth": "10-2000",
    "education": [
        {
            "college_name": "CENTRE FOR DEVELOPMENT OF ADVANCED COMPUTING",
            "stream": "Big Data Analytics",
            "degree_type": "PG Diploma"
        },
        {
            "college_name": "SKN COLLEGE OF ENGINEERING",
            "stream": "Mechanical Engineering",
            "degree_type": "BE"
        }
    ]
}
